# Preprocessing: Water treatment plants

**Objective**: Prepare data from the **MERKUR** dataset for use in machine learning algorithms.

**Background**: The MERKUR project, based in the *Research Centre for Built Environment, Climate, Water Technology and Digitalisation* at VIA University College, collects and analyzes data from water treatment plants in Denmark. In short, the project aims to understand how water treatment plants are run, and the results are then ideally used to optimize the running of water treatment plants. However, the dataset is, as of now, relatively "dirty" in a machine learning context: There are many missing values, outliers, a mix of categorical and numeric data, etc.

**Data Source**: The dataset has kindly been provided to us by Senior Associate Professor Loren Mark Ramsay. You can read more [here](https://en.via.dk/research/built-environment-climate-water-technology-and-digitalisation/water-treatment-and-distribution) and [here](https://www.ucviden.dk/en/projects/merkur-national-web-baseret-dataplatform-til-drikkevandsbehandlin).

Note that we are only working with a subset of the full database. This subset is saved as an Excel file, `merkur.xlsx`.

#### Overall Instructions
1. Explore the dataset to understand the features and their distributions.
2. Preprocess the data, handling any missing values, outliers, etc.

Below some suggestions are given but the assignment is relatively "free".

Best of luck with your analysis!

### Suggestions

-  Filter out (i.e. remove) any irrelevant columns (e.g. names, IDs, etc.)
-  Several columns contain missing values (NaNs). Find out how large a percentage each column is missing. Perhaps some of them lack so much data that you should consider removing them?
-  Scale numeric data.
-  For the features you choose to keep, impute the missing values in an appropriate way - or perhaps you find it more appropriate to delete the rows?
-  Several features (e.g., "PrimaryTrigger") are categorical. Use one-hot encoding to turn them into numeric data. Be careful with the feature "Stages" - perhaps one-hot encoding is not the best choice here?
-  If you you choose to remove or replace outliers, do this now. If you choose to keep, move on.
-  Create a correlation matrix and discuss - based on this, you might want to drop certain columns.
-  Consider whether some features should be transformed (e.g. using log, square root etc.) and do this if found relevant.
-  There are only about 80 rows in the data set. Discuss consequences of this in terms of machine learning - as well as potential solutions. 
-  Think about whether there are other steps you find appropriate at this point. If not, declare your data set clean.

### Data preperation

We start by loading the dataset into a DataFrame:

In [1]:
import pandas as pd

raw_data = pd.read_excel("merkur.xlsx")

raw_data

,WaterworksName,TotalFilters,MaxTypicalFlow,AverageFilterArea,AverageTypicalRunVolume,AverageBackwashVolume,PrimaryTrigger,AverageTotalFilterDepth,OverallFilterGrainSizeMin,OverallFilterGrainSizeMax,...,SumOfld_layer,UniformityCoefficient,UFRV,BW%,HLR_BW,TankCapacity,TankExploitation,GravityPressureMixed,Stages,AbstractedVolume
0,Asnæs Vandværk,6,50.00,6.000000,5200.00000,10.00,Time,NaN,NaN,NaN,...,NaN,NaN,866.666667,0.414525,NaN,27.294778,NaN,Gravity,Single,253543
1,Assens Vandværk,4,110.00,4.908739,300.00000,28.00,Volume,310.0,0.8,5.0,...,2891.666665,NaN,40.743665,23.000062,30.557749,2.576075,NaN,Pressure,Double,612094
2,Astrup Vandværk - Esbjerg,4,120.00,15.343900,3800.00000,83.30,Volume,190.0,2.0,35.0,...,759.000000,1.491,249.217919,2.192086,32.791831,21.825600,NaN,Gravity,Double,682318
3,Astrup Vandværk - Skjern,2,NaN,16.000000,1200.00000,26.00,Volume,NaN,NaN,NaN,...,NaN,NaN,75.000000,2.166907,NaN,12.554300,NaN,Gravity,Single,156998
4,Avernakø Vandværk,2,NaN,1.495000,250.00000,4.50,Volume,NaN,1.6,32.0,...,NaN,NaN,167.224093,1.818182,NaN,52.220566,NaN,Gravity,Single,6710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,Værket ved Regnemark,16,1426.00,47.940002,NaN,110.00,Mix,142.0,0.8,150.0,...,90.809524,NaN,NaN,0.178441,NaN,9.176959,39.682540,Gravity,Double,11454775
75,Værket ved Søndersø,20,1407.29,10.178760,4500.00000,57.00,Volume,190.0,0.8,8.0,...,1708.095230,1.625,442.097088,1.236822,51.086775,6.040393,NaN,Pressure,Single,11021800
76,Østerbyværket,6,250.00,14.752500,2331.50137,51.57,Time,170.0,1.4,4.0,...,842.857154,1.850,158.041107,4.446077,30.367735,22.367525,45.833333,Gravity,Single,423362
77,Østre Vandværk - Herning,12,NaN,37.437401,4400.00000,134.00,Volume,NaN,NaN,NaN,...,NaN,NaN,93.489396,35.386015,32.053507,NaN,NaN,Gravity,Double,1521895


By looking at the data, we have observed that some of the rows had a lot of missing values. To find the distribution of where these missing values are located, we have decided to find the features with the most missing.

In [2]:

# - Overviwe of missing values in the dataset
missing_values = raw_data.isnull().sum()

# - Percentage of missing values in the dataset
missing_percentage = (missing_values / len(raw_data)) * 100

# * Create a new dataframe with the missing values
missing_summary = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage, 'Data Type': raw_data.dtypes})
missing_summary.sort_values(by='Percentage', ascending=False)

,Missing Values,Percentage,Data Type
UniformityCoefficient,51,64.556962,float64
SumOfld_layer,39,49.367089,float64
TotalEBCT,39,49.367089,float64
AverageFilterBedVolume,34,43.037975,float64
OverallFilterGrainSizeMin,34,43.037975,float64
OverallFilterGrainSizeMax,34,43.037975,float64
AverageTotalFilterDepth,33,41.772152,float64
TankExploitation,30,37.974684,float64
HLR_BW,27,34.177215,float64
Stage1HLR,21,26.582278,float64


Based upon this, we can see that columns like `UniformityCoefficient` and `SumOfId_layer` have more than 40% missing values, which might be too much to impute meaningfully.

Columns:
- `UniformityCoeffficient`
- `SumOfId_layer`
- `TotalEBCT`
- `AverageFilterBedVolume`
- `OverallFilterGrainSizeMin`
- `OverallFilterGrainSizeMax`
- `AverageTotalFilterDepth`

For columns with a lower percentage of missing values, we could fill in the missing data using the mean or median for numerical colummns:
- `AverageTypicalRunVolume`
- `AverageFilterArea`
- `Footprint`
- `TankCapacity`
- `UFRV`
- `AverageBackwashVolume`
- `BW%`
- `MaxTypicalFlow`
- `FilterExploitation`
- `Stage1HLR`
- `HLR_BW`
- `TankExplotation`

### Impute missing values

We have decided to impute the values for the columns, which were missing below `40%` of values. `<INSERT SOME REASON HERE THAT MAKES SENSE>`

**Numerical values**
For the columns with numerical data, we opted to impute these missing enttries using the **median** of each respective column. The median, which is the middle value in a list of numbers, is less sensitive to outliers in the data compared to the mean. This would be the perferred choice for imputation in datasets, where outliers might skew the mean siginificantly.

**Categorial values**
For the columns with categorial data, we opted to impute these missing entries using the **mode** of each respective column. The mode is the most frequently occuring value in a data set. This approach is particularly suitable for categorial data since it ensures that the imputed values are actual categories that appear in the data. Also this allows us to preserve the distribution of categories within the dataset.

In [3]:

data_cleaned = raw_data.copy()

# ! Extract the columns with more than 40% missing values
columns_to_drop = missing_summary[missing_summary['Percentage'] > 40]

data_cleaned = data_cleaned.drop(columns=columns_to_drop.index)

# ! Drop irrelevant columns like `WaterworksName`
data_cleaned = data_cleaned.drop(columns=['WaterworksName'])

# * Impute missing values for numerical columns using the median
numerical_columns = data_cleaned.select_dtypes(include=['float64','int64']).columns
data_cleaned[numerical_columns] = data_cleaned[numerical_columns].fillna(data_cleaned[numerical_columns].median())

# * Impute missing values for categorical columns using the mode
categorical_columns = data_cleaned.select_dtypes(include=['object']).columns

for column in categorical_columns:
    mode_value = data_cleaned[column].mode()[0]
    data_cleaned[column] = data_cleaned[column].fillna(mode_value)

# - Check for missing values after imputation
missing_values_after_imputation = data_cleaned.isnull().sum().sum()


# * Show the first 5 rows of the cleaned dataset
data_cleaned.head()


,TotalFilters,MaxTypicalFlow,AverageFilterArea,AverageTypicalRunVolume,AverageBackwashVolume,PrimaryTrigger,FilterExploitation,AerationType,OxygenFactor,Stage1HLR,Footprint,UFRV,BW%,HLR_BW,TankCapacity,TankExploitation,GravityPressureMixed,Stages,AbstractedVolume
0,6,50.0,6.000000,5200.0,10.0,Time,57.886530,Cascade,1.880000,1.388889,143.341204,866.666667,0.414525,30.889453,27.294778,19.402985,Gravity,Single,253543
1,4,110.0,4.908739,300.0,28.0,Volume,63.521586,Air injection,1.620000,11.204508,16.714412,40.743665,23.000062,30.557749,2.576075,19.402985,Pressure,Double,612094
2,4,120.0,15.343900,3800.0,83.3,Volume,64.908486,Bottom aeration,2.213333,3.935020,44.693822,249.217919,2.192086,32.791831,21.825600,19.402985,Gravity,Double,682318
3,2,120.0,16.000000,1200.0,26.0,Volume,63.279585,Cascade,2.073333,3.742502,194.588021,75.000000,2.166907,30.889453,12.554300,19.402985,Gravity,Single,156998
4,2,120.0,1.495000,250.0,4.5,Volume,63.279585,Cascade,1.313333,3.742502,412.300024,167.224093,1.818182,30.889453,52.220566,19.402985,Gravity,Single,6710


After we have imputed the dataset, we want to check that there are no missing values left in the dataset. We can do this by running the following code:

In [4]:
data_cleaned.isna().sum()

TotalFilters               0
MaxTypicalFlow             0
AverageFilterArea          0
AverageTypicalRunVolume    0
AverageBackwashVolume      0
PrimaryTrigger             0
FilterExploitation         0
AerationType               0
OxygenFactor               0
Stage1HLR                  0
Footprint                  0
UFRV                       0
BW%                        0
HLR_BW                     0
TankCapacity               0
TankExploitation           0
GravityPressureMixed       0
Stages                     0
AbstractedVolume           0
dtype: int64

### Removing outliers

Considering the significant impact that outliers can have on the performance of machine learning algorithms, we've decided to prioritize the quality of our data over quantity. Therefore, we have chosen to remove any rows containing outliers in any of the features that could potentially affect our model's accuracy.

One might question the decision to eliminate an entire row based on a single outlier. An alternative approach could involve tallying the number of outliers per row and only removing those rows exceeding a certain threshold. However, given our dataset's limited size—which could hamper the accuracy of a machine learning model—removing outliers seems to be the most prudent strategy to enhance the reliability of our future predictions.

In [5]:
from scipy import stats
import numpy as np

irrelevant_columns = ['TotalFilters', 'TankCapacity', 'OxygenFactor']
relevant_columns = [col for col in numerical_columns if col not in irrelevant_columns]

# Calculate the z-score for all values in relevant columns
for col in relevant_columns:
    z_scores = stats.zscore(data_cleaned[col])
    abs_z_scores = np.abs(z_scores)
    filtered_entries = (abs_z_scores < 3)  # Adjust 3 as per requirement
    data_cleaned = data_cleaned[filtered_entries]

# Reset index
data_cleaned.reset_index(drop=False, inplace=True)

# Now, data_cleaned has only the filtered entries with a new continuous index
data_cleaned.head()

,index,TotalFilters,MaxTypicalFlow,AverageFilterArea,AverageTypicalRunVolume,AverageBackwashVolume,PrimaryTrigger,FilterExploitation,AerationType,OxygenFactor,Stage1HLR,Footprint,UFRV,BW%,HLR_BW,TankCapacity,TankExploitation,GravityPressureMixed,Stages,AbstractedVolume
0,0,6,50.0,6.000000,5200.000000,10.0,Time,57.886530,Cascade,1.880000,1.388889,143.341204,866.666667,0.414525,30.889453,27.294778,19.402985,Gravity,Single,253543
1,2,4,120.0,15.343900,3800.000000,83.3,Volume,64.908486,Bottom aeration,2.213333,3.935020,44.693822,249.217919,2.192086,32.791831,21.825600,19.402985,Gravity,Double,682318
2,3,2,120.0,16.000000,1200.000000,26.0,Volume,63.279585,Cascade,2.073333,3.742502,194.588021,75.000000,2.166907,30.889453,12.554300,19.402985,Gravity,Single,156998
3,5,12,215.0,14.299999,2833.333333,30.0,Volume,70.189020,Cascade,1.786667,1.879371,86.539475,148.601406,1.411789,17.482518,26.506498,10.285714,Gravity,Double,1321940
4,6,5,51.0,4.000000,870.000000,10.5,Volume,40.547721,Cascade,1.406667,12.750000,20.232778,87.500000,4.050212,30.889453,4.835745,35.714286,Gravity,Double,181151


After removing the outliers, we have decided upon resetting the intial index, and save the original index as a part of the dataframe.

### Scale the numeric

We focused on scaling the numeric features of our dataset to ensure consistency across all input variables. This is crucial since numeric columns with varying scales can disproportionately influence machine learning models, potentially leading to biased results. We employed the MinMaxScaler from the sklearn.preprocessing module, which transforms each feature to a given range, usually 0 to 1.

In [6]:
from sklearn.preprocessing import MinMaxScaler

# Declare the columns which are relevant for scaling
irrelevant_columns = ['index'] # It wouldn't make sense to scale the original index.
relevant_columns = [col for col in numerical_columns if col not in irrelevant_columns]

# Scale the relevant columns
scaler = MinMaxScaler()
scaled = scaler.fit_transform(data_cleaned[relevant_columns])
scaled_df = pd.DataFrame(scaled, columns=relevant_columns)

# Add the original index to the scaled dataframe
scaled_df['Org Index'] = data_cleaned['index']

# Show the first 5 rows of the scaled dataframe
scaled_df.head()

,TotalFilters,MaxTypicalFlow,AverageFilterArea,AverageTypicalRunVolume,AverageBackwashVolume,FilterExploitation,OxygenFactor,Stage1HLR,Footprint,UFRV,BW%,HLR_BW,TankCapacity,TankExploitation,AbstractedVolume,Org Index
0,0.294118,0.062464,0.193906,0.225012,0.100029,0.393157,0.610390,0.038377,0.672509,0.973378,0.026832,0.587067,0.261953,0.337714,0.064922,0
1,0.176471,0.162981,0.581657,0.160430,0.896999,0.464762,0.880952,0.206473,0.181051,0.233672,0.166777,0.637074,0.203570,0.337714,0.197452,2
2,0.058824,0.162981,0.608883,0.040491,0.273992,0.448152,0.767316,0.193763,0.927819,0.024958,0.164795,0.587067,0.104600,0.337714,0.035081,3
3,0.647059,0.299397,0.538337,0.115837,0.317483,0.518609,0.534632,0.070759,0.389525,0.113133,0.105345,0.234644,0.253538,0.155996,0.395153,5
4,0.235294,0.063900,0.110910,0.025268,0.105465,0.216348,0.226190,0.788443,0.059187,0.039933,0.313066,0.587067,0.022205,0.662818,0.042547,6


After scaling the numerical columns, we want to re-attach the categorial columns to the dataframe:

In [7]:

# - combine the scaled dataframe with the original categorical columns
combined_df = pd.concat([scaled_df, data_cleaned[categorical_columns]], axis=1)

# - Show the first 5 rows of the combined dataframe
combined_df.head()

,TotalFilters,MaxTypicalFlow,AverageFilterArea,AverageTypicalRunVolume,AverageBackwashVolume,FilterExploitation,OxygenFactor,Stage1HLR,Footprint,UFRV,BW%,HLR_BW,TankCapacity,TankExploitation,AbstractedVolume,Org Index,PrimaryTrigger,AerationType,GravityPressureMixed,Stages
0,0.294118,0.062464,0.193906,0.225012,0.100029,0.393157,0.610390,0.038377,0.672509,0.973378,0.026832,0.587067,0.261953,0.337714,0.064922,0,Time,Cascade,Gravity,Single
1,0.176471,0.162981,0.581657,0.160430,0.896999,0.464762,0.880952,0.206473,0.181051,0.233672,0.166777,0.637074,0.203570,0.337714,0.197452,2,Volume,Bottom aeration,Gravity,Double
2,0.058824,0.162981,0.608883,0.040491,0.273992,0.448152,0.767316,0.193763,0.927819,0.024958,0.164795,0.587067,0.104600,0.337714,0.035081,3,Volume,Cascade,Gravity,Single
3,0.647059,0.299397,0.538337,0.115837,0.317483,0.518609,0.534632,0.070759,0.389525,0.113133,0.105345,0.234644,0.253538,0.155996,0.395153,5,Volume,Cascade,Gravity,Double
4,0.235294,0.063900,0.110910,0.025268,0.105465,0.216348,0.226190,0.788443,0.059187,0.039933,0.313066,0.587067,0.022205,0.662818,0.042547,6,Volume,Cascade,Gravity,Double


Now to make the data more readable, we have decided to reorder the colunms, before continuing. (Since we want to have the Original index as the first column.)

In [8]:

# Utility function - Set the column order to be more intuitive
def reorder_columns(data : pd.DataFrame) -> pd.DataFrame:
    # ? Reordering the columns
    # So the columns are easier to understand and work with.
    strict_column_order = ['Org Index']
    remaining_columns = [col for col in data.columns if col not in strict_column_order]
    data = data[strict_column_order + remaining_columns]
    return data

# Reorder the columns
combined_df = reorder_columns(combined_df)

# Show the first 5 rows of the reordered dataframe
combined_df.head()


,Org Index,TotalFilters,MaxTypicalFlow,AverageFilterArea,AverageTypicalRunVolume,AverageBackwashVolume,FilterExploitation,OxygenFactor,Stage1HLR,Footprint,UFRV,BW%,HLR_BW,TankCapacity,TankExploitation,AbstractedVolume,PrimaryTrigger,AerationType,GravityPressureMixed,Stages
0,0,0.294118,0.062464,0.193906,0.225012,0.100029,0.393157,0.610390,0.038377,0.672509,0.973378,0.026832,0.587067,0.261953,0.337714,0.064922,Time,Cascade,Gravity,Single
1,2,0.176471,0.162981,0.581657,0.160430,0.896999,0.464762,0.880952,0.206473,0.181051,0.233672,0.166777,0.637074,0.203570,0.337714,0.197452,Volume,Bottom aeration,Gravity,Double
2,3,0.058824,0.162981,0.608883,0.040491,0.273992,0.448152,0.767316,0.193763,0.927819,0.024958,0.164795,0.587067,0.104600,0.337714,0.035081,Volume,Cascade,Gravity,Single
3,5,0.647059,0.299397,0.538337,0.115837,0.317483,0.518609,0.534632,0.070759,0.389525,0.113133,0.105345,0.234644,0.253538,0.155996,0.395153,Volume,Cascade,Gravity,Double
4,6,0.235294,0.063900,0.110910,0.025268,0.105465,0.216348,0.226190,0.788443,0.059187,0.039933,0.313066,0.587067,0.022205,0.662818,0.042547,Volume,Cascade,Gravity,Double


### One Hot encoding of categorial columns

As the next step to having a dataset, that is ready to be used for machine learning. We have decided to one hot encode the categorial columns.

The reason we one hot encode here is because it makes it easier for a classification model to process.
It also increases the quality of the model in the end, because the model is able to capture subtle distinctions between the different categories.

In [9]:

# - One hot encode the categorical columns
one_hot_encoded = pd.get_dummies(combined_df, columns=categorical_columns)

# - Show the first 5 rows of the one hot encoded dataframe
one_hot_encoded.head()

,Org Index,TotalFilters,MaxTypicalFlow,AverageFilterArea,AverageTypicalRunVolume,AverageBackwashVolume,FilterExploitation,OxygenFactor,Stage1HLR,Footprint,...,AerationType_Other,AerationType_Passive plate aerator,AerationType_Pure oxygen injection,GravityPressureMixed_Gravity,GravityPressureMixed_Mixed,GravityPressureMixed_Pressure,Stages_Double,Stages_Mixed,Stages_Single,Stages_Triple
0,0,0.294118,0.062464,0.193906,0.225012,0.100029,0.393157,0.610390,0.038377,0.672509,...,False,False,False,True,False,False,False,False,True,False
1,2,0.176471,0.162981,0.581657,0.160430,0.896999,0.464762,0.880952,0.206473,0.181051,...,False,False,False,True,False,False,True,False,False,False
2,3,0.058824,0.162981,0.608883,0.040491,0.273992,0.448152,0.767316,0.193763,0.927819,...,False,False,False,True,False,False,False,False,True,False
3,5,0.647059,0.299397,0.538337,0.115837,0.317483,0.518609,0.534632,0.070759,0.389525,...,False,False,False,True,False,False,True,False,False,False
4,6,0.235294,0.063900,0.110910,0.025268,0.105465,0.216348,0.226190,0.788443,0.059187,...,False,False,False,True,False,False,True,False,False,False


### Thoughts for consideration

Due to insufficient materials for a thorough understanding of the domain, we may have inadvertently eliminated columns that were significant within the domain. This potential oversight, coupled with the excessive removal and imputation of rows in the original dataset, has left us without a clear reference for the relevance of various data points within the domain context.

Operating with a significantly reduced dataset presents challenges in machine learning. Minor discrepancies can disproportionately impact the model's accuracy due to the limited data available to offset such variations. Furthermore, the necessity to allocate the majority of data for training exacerbates the issue, as it restricts the amount available for testing.

The lack of comprehensive domain knowledge compounds these challenges, increasing the risk of discarding vital information from our dataset. Without a solid understanding, there is a higher likelihood of excessive deletion or imputation, complicating our ability to interpret the data meaningfully.